<h1> Using Labeled Data to Train a Network, Use The Network to Label New Videos, and Create Trajectories </h1> 
<p> Use another notebook create a project, extract frames, and label training/testing data </p>

# DLC with docker
This notebook illustrates how to use the Docker container to:
- train a network
- evaluate a network
- analyze a novel video

many of the functions have additional parameters / complexity, see the DLC docs for more inf on each.

This assumes you already have a project folder with labeled data! 


## info about the Docker Environment:
Docker is essentially a better alternative to virtual machines. It is able to containerize applications in a way that keeps them seperate the OS and other software. 

For tensorflow / deeplab cut, docker is critical for two reasons. 
- It 'claims' the GPU, making it closed for use by other processes.
- It queries the GPU properly, in some cases tf (and the python kernal itself) does not know how to properly communincate with the GPU on windows systems.

In [1]:
# make sure you graphic driver is accessable
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2017 NVIDIA Corporation
Built on Fri_Sep__1_21:08:32_Central_Daylight_Time_2017
Cuda compilation tools, release 9.0, V9.0.176


In [2]:
# check the tensorflow version
import tensorflow as tf
tf.__version__

'1.14.0'

<h1> The following is very important as it allow the current GPU process to grow dynamically </h1>
Without this option tf will likely run out of VRAM when trying to update the weight tensor. In theory, these options could cause the GPU to run out of memory entirely, but there is no other way to allow training to complete successfully. 

In [3]:
#allow video memory growth as network expands to avoid convolutional network errors
TF_FORCE_GPU_ALLOW_GROWTH = True
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [4]:
#let's make sure we see a GPU:
#tf.test.gpu_device_name()
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13537146552544263055,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 6586089472
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 17967924156679516769
 physical_device_desc: "device: 0, name: GeForce RTX 2080, pci bus id: 0000:41:00.0, compute capability: 7.5",
 name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 6586089472
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13782940445016201425
 physical_device_desc: "device: 1, name: GeForce RTX 2080, pci bus id: 0000:81:00.0, compute capability: 7.5"]

## Start here for training DeepLabCut and analyzing new videos!
<p><br>If the first imports fail, there is again - sadly - an issue with you enviroment. Make sure all packages beside DLC are installed via conda. </p> 

In [5]:
#GUIs don't work on in Docker (or the cloud), so label your data locally on your computer! 
#This notebook is for you to train and run video analysis!
import os

In [6]:
# now we are ready to train!
#should see version 2.0.8
import deeplabcut
deeplabcut.__version__

'2.1.10.1'

<h1> change to your path: </h1>
<p> this should be the same path as the one in the createDLCproject notebook. The path is the path to the config.yaml file, not the project directory itself </p> 

In [7]:
path_config_file = r'F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam1\box1_cam1-spencerloggia-2021-03-18\config.yaml'


## Create a training dataset
This function generates the training data information for DeepCut (which requires a mat file) based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles. 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

In [8]:
deeplabcut.create_training_dataset(path_config_file, net_type='resnet_50', augmenter_type='imgaug')

F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam1\box1_cam1-spencerloggia-2021-03-18\training-datasets\iteration-0\UnaugmentedDataSet_box1_cam1Mar18  already exists!
failed on F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam1\box1_cam1-spencerloggia-2021-03-18\labeled-data\camera-1_clip-7\CollectedData_spencerloggia.h5
failed on F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam1\box1_cam1-spencerloggia-2021-03-18\labeled-data\camera-1_clip-57\CollectedData_spencerloggia.h5
failed on F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam1\box1_cam1-spencerloggia-2021-03-18\labeled-data\camera-1_clip-135\CollectedData_spencerloggia.h5
F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam1\box1_cam1-spencerloggia-2021-03-18\dlc-models\iteration-0\box1_cam1Mar18-trainset95shuffle1  already exists!
F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam1\box1_cam1-spencerloggia-2021-03-18\dlc-models\iteration-0\box1_cam1Mar18-trainset95shuffle1/train  already exists!
F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam1\box1_cam1-spe

[(0.95,
  1,
  (array([ 65, 123, 172,  85, 147,  50,  41,  52, 167,  81, 129, 191, 130,
          132, 127, 159,  54,  44,  59, 150,  10, 201, 165,  99, 105, 182,
           63, 103,  93, 175, 134,   3, 109,  15,  37,  27, 139, 108,  35,
          157, 146, 169,  76,  22,  61, 102,  16, 200, 152, 133,  45,  29,
          192,  32, 112,  69, 120,  70, 164, 153, 135,  75,   6,  71,  90,
          100,   7, 181, 111,  97, 197,  64,   4, 114,  38, 184,  21, 180,
           87, 106,  19,  80, 124,   9,  62, 162, 185, 115, 117, 158,  25,
            1, 195,  51, 125,  83,  13,  36,  96,  82,  55,  20, 190, 193,
          122, 204,  67,  34, 189,  11,  28,  66, 203,   5,  98,  48, 149,
          186, 187, 174, 171,  91,  78, 179,  89, 183, 126,  12, 170, 202,
          173,  72, 154, 131,  68, 163,  24, 161,  73, 188, 138, 110,  86,
          121,  31,  40,  79, 143,  33,  17, 156, 136, 141, 116,   8,  47,
          151,  26, 144,  56, 113,  84, 178,  94,  58, 196, 166, 140,  23,
           7

### now go edit the pose_cfg.yaml to make display_iters: low (i.e. 10), and save_iters: 500

Now it is the time to start training the network!

## Start training
This function trains the network for a specific shuffle of the training dataset. 

In [10]:
#reset in case you started a session before...
#tf.reset_default_graph()

deeplabcut.train_network(path_config_file, shuffle=1, saveiters=1000, displayiters=10, gputouse=0)

#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 1.3M iterations). 
#Whichever you chose, you will see what looks like an error message, but it's not an error - don't worry....


Config:
{'all_joints': [[0], [1], [2]],
 'all_joints_names': ['bodypart1', 'bodypart2', 'bodypart3'],
 'alpha_r': 0.02,
 'batch_size': 1,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets\\iteration-0\\UnaugmentedDataSet_box1_cam1Mar18\\box1_cam1_spencerloggia95shuffle1.mat',
 'dataset_type': 'imgaug',
 'decay_steps': 30000,
 'deterministic': False,
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': 'f:\\mysoredata\\nbk\\mousevideoanalysis\\dlc_env_conda\\lib\\site-packages\\deeplabcut\\pose_estimation_tensorflow\\models\\pretrained\\resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'lr_init': 0.0005,
 'max_input_size': 1500,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets\\iteration-0\\UnaugmentedDataSet_box1_cam1Mar18\\Documentatio

Selecting single-animal trainer
Starting with imgaug pose-dataset loader (=default).
Batch Size is 1
Initializing ResNet


Loading ImageNet-pretrained resnet_50
Display_iters overwritten as 10
Save_iters overwritten as 1000
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': 'F:\\MysoreData\\nbk\\mouseVideoAnalysis\\Box1\\cam1\\box1_cam1-spencerloggia-2021-03-18\\dlc-models\\iteration-0\\box1_cam1Mar18-trainset95shuffle1\\train\\snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'crop_pad': 0, 'scoremap_dir': 'test', 'batch_size': 1, 'dataset_type': 'imgaug', 'deterministic': False, 'mirror': False, 'pairwise_huber_loss': False, 'weigh_only_present_joints': False, 'partaffinityfield_predict': False, 'pairwise_predict': False, 

iteration: 10 loss: 0.2430 lr: 0.005
iteration: 20 loss: 0.0564 lr: 0.005
iteration: 30 loss: 0.0407 lr: 0.005
iteration: 40 loss: 0.0330 lr: 0.005
iteration: 50 loss: 0.0390 lr: 0.005
iteration: 60 loss: 0.0361 lr: 0.005
iteration: 70 loss: 0.0277 lr: 0.005
iteration: 80 loss: 0.0390 lr: 0.005
iteration: 90 loss: 0.0259 lr: 0.005
iteration: 100 loss: 0.0232 lr: 0.005
iteration: 110 loss: 0.0315 lr: 0.005
iteration: 120 loss: 0.0224 lr: 0.005
iteration: 130 loss: 0.0240 lr: 0.005
iteration: 140 loss: 0.0208 lr: 0.005
iteration: 150 loss: 0.0240 lr: 0.005
iteration: 160 loss: 0.0204 lr: 0.005
iteration: 170 loss: 0.0270 lr: 0.005
iteration: 180 loss: 0.0206 lr: 0.005
iteration: 190 loss: 0.0192 lr: 0.005
iteration: 200 loss: 0.0203 lr: 0.005
iteration: 210 loss: 0.0251 lr: 0.005
iteration: 220 loss: 0.0227 lr: 0.005
iteration: 230 loss: 0.0202 lr: 0.005
iteration: 240 loss: 0.0187 lr: 0.005
iteration: 250 loss: 0.0206 lr: 0.005
iteration: 260 loss: 0.0208 lr: 0.005
iteration: 270 loss: 

iteration: 2140 loss: 0.0139 lr: 0.005
iteration: 2150 loss: 0.0150 lr: 0.005
iteration: 2160 loss: 0.0138 lr: 0.005
iteration: 2170 loss: 0.0137 lr: 0.005
iteration: 2180 loss: 0.0155 lr: 0.005
iteration: 2190 loss: 0.0125 lr: 0.005
iteration: 2200 loss: 0.0160 lr: 0.005
iteration: 2210 loss: 0.0170 lr: 0.005
iteration: 2220 loss: 0.0140 lr: 0.005
iteration: 2230 loss: 0.0156 lr: 0.005
iteration: 2240 loss: 0.0120 lr: 0.005
iteration: 2250 loss: 0.0142 lr: 0.005
iteration: 2260 loss: 0.0187 lr: 0.005
iteration: 2270 loss: 0.0105 lr: 0.005
iteration: 2280 loss: 0.0131 lr: 0.005
iteration: 2290 loss: 0.0142 lr: 0.005
iteration: 2300 loss: 0.0131 lr: 0.005
iteration: 2310 loss: 0.0111 lr: 0.005
iteration: 2320 loss: 0.0128 lr: 0.005
iteration: 2330 loss: 0.0147 lr: 0.005
iteration: 2340 loss: 0.0136 lr: 0.005
iteration: 2350 loss: 0.0124 lr: 0.005
iteration: 2360 loss: 0.0129 lr: 0.005
iteration: 2370 loss: 0.0178 lr: 0.005
iteration: 2380 loss: 0.0152 lr: 0.005
iteration: 2390 loss: 0.0

...

iteration: 507290 loss: 0.0009 lr: 0.002
iteration: 507300 loss: 0.0011 lr: 0.002
iteration: 507310 loss: 0.0013 lr: 0.002
iteration: 507320 loss: 0.0007 lr: 0.002
iteration: 507330 loss: 0.0007 lr: 0.002
iteration: 507340 loss: 0.0009 lr: 0.002
iteration: 507350 loss: 0.0013 lr: 0.002
iteration: 507360 loss: 0.0006 lr: 0.002
iteration: 507370 loss: 0.0013 lr: 0.002
iteration: 507380 loss: 0.0012 lr: 0.002
iteration: 507390 loss: 0.0008 lr: 0.002
iteration: 507400 loss: 0.0009 lr: 0.002
iteration: 507410 loss: 0.0014 lr: 0.002
iteration: 507420 loss: 0.0011 lr: 0.002
iteration: 507430 loss: 0.0015 lr: 0.002
iteration: 507440 loss: 0.0014 lr: 0.002
iteration: 507450 loss: 0.0011 lr: 0.002
iteration: 507460 loss: 0.0014 lr: 0.002
iteration: 507470 loss: 0.0021 lr: 0.002
iteration: 507480 loss: 0.0016 lr: 0.002
iteration: 507490 loss: 0.0019 lr: 0.002
iteration: 507500 loss: 0.0008 lr: 0.002
iteration: 507510 loss: 0.0013 lr: 0.002
iteration: 507520 loss: 0.0009 lr: 0.002
iteration: 50753

iteration: 509290 loss: 0.0008 lr: 0.002
iteration: 509300 loss: 0.0014 lr: 0.002
iteration: 509310 loss: 0.0010 lr: 0.002
iteration: 509320 loss: 0.0016 lr: 0.002
iteration: 509330 loss: 0.0009 lr: 0.002
iteration: 509340 loss: 0.0008 lr: 0.002
iteration: 509350 loss: 0.0019 lr: 0.002
iteration: 509360 loss: 0.0008 lr: 0.002
iteration: 509370 loss: 0.0012 lr: 0.002
iteration: 509380 loss: 0.0010 lr: 0.002
iteration: 509390 loss: 0.0014 lr: 0.002
iteration: 509400 loss: 0.0011 lr: 0.002
iteration: 509410 loss: 0.0012 lr: 0.002
iteration: 509420 loss: 0.0010 lr: 0.002
iteration: 509430 loss: 0.0010 lr: 0.002
iteration: 509440 loss: 0.0014 lr: 0.002
iteration: 509450 loss: 0.0010 lr: 0.002
iteration: 509460 loss: 0.0014 lr: 0.002
iteration: 509470 loss: 0.0013 lr: 0.002
iteration: 509480 loss: 0.0015 lr: 0.002
iteration: 509490 loss: 0.0015 lr: 0.002
iteration: 509500 loss: 0.0018 lr: 0.002
iteration: 509510 loss: 0.0013 lr: 0.002
iteration: 509520 loss: 0.0016 lr: 0.002
iteration: 50953

iteration: 511290 loss: 0.0010 lr: 0.002
iteration: 511300 loss: 0.0018 lr: 0.002
iteration: 511310 loss: 0.0010 lr: 0.002
iteration: 511320 loss: 0.0019 lr: 0.002
iteration: 511330 loss: 0.0013 lr: 0.002
iteration: 511340 loss: 0.0017 lr: 0.002
iteration: 511350 loss: 0.0018 lr: 0.002
iteration: 511360 loss: 0.0012 lr: 0.002
iteration: 511370 loss: 0.0012 lr: 0.002
iteration: 511380 loss: 0.0009 lr: 0.002
iteration: 511390 loss: 0.0012 lr: 0.002
iteration: 511400 loss: 0.0012 lr: 0.002
iteration: 511410 loss: 0.0009 lr: 0.002
iteration: 511420 loss: 0.0011 lr: 0.002
iteration: 511430 loss: 0.0012 lr: 0.002
iteration: 511440 loss: 0.0012 lr: 0.002
iteration: 511450 loss: 0.0011 lr: 0.002
iteration: 511460 loss: 0.0013 lr: 0.002
iteration: 511470 loss: 0.0013 lr: 0.002
iteration: 511480 loss: 0.0014 lr: 0.002
iteration: 511490 loss: 0.0009 lr: 0.002
iteration: 511500 loss: 0.0009 lr: 0.002
iteration: 511510 loss: 0.0009 lr: 0.002
iteration: 511520 loss: 0.0011 lr: 0.002
iteration: 51153

iteration: 513290 loss: 0.0012 lr: 0.002
iteration: 513300 loss: 0.0011 lr: 0.002
iteration: 513310 loss: 0.0014 lr: 0.002
iteration: 513320 loss: 0.0022 lr: 0.002
iteration: 513330 loss: 0.0013 lr: 0.002
iteration: 513340 loss: 0.0016 lr: 0.002
iteration: 513350 loss: 0.0013 lr: 0.002
iteration: 513360 loss: 0.0011 lr: 0.002
iteration: 513370 loss: 0.0012 lr: 0.002
iteration: 513380 loss: 0.0011 lr: 0.002


KeyboardInterrupt: 

### A couple tips for possible troubleshooting (1): 

if you get **permission errors** when you run this step (above), first check if the weights downloaded. As some docker containers might not have privileges for this (it can be user specific). They should be under 'init_weights' (see path in the pose_cfg.yaml file). You can enter the DOCKER in the terminal:

see more here: https://github.com/MMathisLab/Docker4DeepLabCut2.0#using-the-docker-for-training-and-video-analysis

### Troubleshooting (2): 
if it appears the training does not start (i.e. "Starting training..." does not print immediately),
then you have another session running on your GPU. Go check "nvidia-smi" and look at the process names. You can only have 1 per GPU!)

## Start evaluating
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [11]:
deeplabcut.evaluate_network(path_config_file)

# Here you want to see a low pixel error! Of course, it can only be as good as the labeler, so be sure your labels are good!

Config:
{'all_joints': [[0], [1], [2]],
 'all_joints_names': ['bodypart1', 'bodypart2', 'bodypart3'],
 'batch_size': 1,
 'crop_pad': 0,
 'dataset': 'training-datasets\\iteration-0\\UnaugmentedDataSet_box1_cam1Mar18\\box1_cam1_spencerloggia95shuffle1.mat',
 'dataset_type': 'imgaug',
 'deterministic': False,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': 'f:\\mysoredata\\nbk\\mousevideoanalysis\\dlc_env_conda\\lib\\site-packages\\deeplabcut\\pose_estimation_tensorflow\\models\\pretrained\\resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 1.0,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'mean_pixel': [123.68, 116.779, 103.939],
 'mirror': False,
 'net_type': 'resnet_50',
 'num_joints': 3,
 'optimizer': 'sgd',
 'pairwise_huber_loss': True,
 'pairwise_predict': False,
 'partaffinityfield_predict': False,
 'regularize': False,
 'scoremap_dir': 'test',
 

Running  DLC_resnet_50_box1_cam1Mar18shuffle1_513000  with # of trainingiterations: 513000
Initializing ResNet


0it [00:00, ?it/s]

Analyzing data...


205it [00:22,  9.03it/s]


Done and results stored for snapshot:  snapshot-513000
Results for 513000  training iterations: 95 1 train error: 1.63 pixels. Test error: 17.13  pixels.
With pcutoff of 0.6  train error: 1.63 pixels. Test error: 8.04 pixels
Thereby, the errors are given by the average distances between the labels by DLC and the scorer.
The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
If it generalizes well, choose the best model for prediction and update the config file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise consider retraining the network (see DeepLabCut workflow Fig 2)


## There is an optional refinement step
- if your pixel errors are not low enough, use DLC docs on how to refine yur network!
- You will need to adjust the labels outside of DOCKER! (you can use the createDLCproject notebook) 
-  see DLC protocol instructions on how to refine your data!

## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [ ]:

VIDEO_SOURCE = r"F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam1\test_labels"
video_all = os.listdir(VIDEO_SOURCE)
video = []
for i in range(0,len(video_all)):
    video.append(VIDEO_SOURCE + '\\' + video_all[i])
    
deeplabcut.analyze_videos(path_config_file,video)

## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [ ]:
deeplabcut.create_labeled_video(path_config_file,video)

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
%matplotlib notebook 
#for making interactive plots.
#deeplabcut.plot_trajectories(path_config_file,videofile_path, plotting=True)

deeplabcut.plot_trajectories(path_config_file,video,showfigures=True)